# Deep Dive into LangChain
## Tools and Agents + ReAct

This notebook uses the latest versions of the OpenAI and LangChain libraries.

In [1]:
# authenticating to OpenAI
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys from .env
load_dotenv(find_dotenv(), override=True)

True

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys from .env
load_dotenv(find_dotenv(), override=True)

# os.environ.get('OPENAI_API_KEY')

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()  

# invoking the llm (running the prompt)
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', temperature=0.1)
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of particles at the smallest scales, where traditional laws of physics no longer apply and phenomena such as superposition and entanglement occur.


In [4]:
# help(ChatOpenAI)  # see the llm constructor arguments with its defaults

In [5]:
# using Chat Completions API Messages: System, Assistant and Human

from langchain.schema import(
    SystemMessage, 
    AIMessage,
    HumanMessage
)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik ist die physikalische Theorie, die das Verhalten von Teilchen auf atomarer und subatomarer Ebene beschreibt, indem sie auf Wahrscheinlichkeitsverteilungen und quantenmechanische Zustände zurückgreift.


In [6]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: user 12.7 ms, sys: 2.57 ms, total: 15.2 ms
Wall time: 930 ms


AIMessage(content='Why did the cookie go to the doctor? Because it was feeling crumbly!', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 18, 'total_tokens': 35}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-36571e13-dbe1-484b-bd76-97a71ae069d4-0', usage_metadata={'input_tokens': 18, 'output_tokens': 17, 'total_tokens': 35})

In [7]:
%%time
llm.invoke(prompt)

CPU times: user 11.6 ms, sys: 2.23 ms, total: 13.8 ms
Wall time: 1.09 s


AIMessage(content="Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired!", response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 18, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a404b9a8-feb8-4294-b269-78f978d0eb9d-0', usage_metadata={'input_tokens': 18, 'output_tokens': 16, 'total_tokens': 34})

## LangChain Agents in Action: Python REPL

In [8]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [9]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with gpt-4-turbo and a temperature of 0
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
First, I need to calculate the factorial of 12. Then, I will find the square root of that result. Finally, I will format the output to display it with 4 decimal points.
Action: Python_REPL
Action Input: import math
print(f"{math.sqrt(math.factorial(12)):.4f}")
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [10]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I will calculate the power of 5.1 raised to 7.3 using Python.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [11]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

In [12]:
print(response['input'])

What is the answer to 5.1 ** 7.3?


In [13]:
print(response['output'])


146306.05007233328


## LangChain Tools: DuckDuckGo and Wikipedia

### DuckDuckGo Search

In [1]:
# optional, filter workings (recommended in production)
import warnings
warnings.filterwarnings('ignore', module='langchain')

In [2]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke('Where was Freddie Mercury born?')
print(output)

Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock's most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar ... Freddie Mercury was born Farrokh Bulsara in Stone Town, in the British protectorate of Zanzibar (now part of Tanzania), on September 5, 1946. His first big challenge was to come to terms with ... But there are many lesser-known Freddie Mercury facts that reveal hidden depths to the life and work of the man born Farrokh Bulsara in Zanzibar, on September 5, 1946. Freddie Mercury was Britain's first Indian rock star. This fact and the nature of his sexuality were the two areas of his short life about which he would be intentionally obscure. Although he was born on the East African island of Zanzibar on 

In [3]:
search.name

'duckduckgo_search'

In [4]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [5]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
output = search.run('Freddie Mercury and Queen.')
print(output)

[snippet: Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock's most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar ..., title: Freddie Mercury | Biography, Parents, Songs, & Facts, link: https://www.britannica.com/biography/Freddie-Mercury], [snippet: Freddie Mercury always talked about his pride at writing "Killer Queen," which appeared on Queen's 1974 album Sheer Heart Attack. He said it was written in one night and was a song that he ..., title: Best Freddie Mercury Songs: 20 Essential Solo And Queen Tracks, link: https://www.udiscovermusic.com/stories/best-freddie-mercury-songs/], [snippet: If you're going to be the greatest live band in the world, having one of the greatest frontmen in rock and roll hi

In [6]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region='de-de', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Berlin')

In [7]:
print(output)

[snippet: Berlin [bɛr'li:n] ist die Hauptstadt und ein Land der Bundesrepublik Deutschland. Die Großstadt ist mit rund 3,8 Millionen Einwohnern die bevölkerungsreichste und mit 892 Quadratkilometern die flächengrößte Gemeinde Deutschlands sowie die bevölkerungsreichste Stadt der Europäischen Union., title: Berlin - Wikipedia, link: https://de.wikipedia.org/wiki/Berlin], [snippet: 31. Mai bis 02. Juni 2024: Das Berlin-Programm für das Wochenende in der Hauptstadt., title: Wochenend-Tipps: Endlich Wochenende in Berlin! - Berlin.de, link: https://www.berlin.de/wochenend-tipps/], [snippet: Berlin in 7 Tagen - Die besten Berlin Sehenswürdigkeiten, Tipps, Highlights, Geheimtipps und Empfehlungen zu Hotels und Restaurants., title: Berlin Sehenswürdigkeiten: Die 41 schönsten Orte | in 3 - 7 Tagen, link: https://wetraveltheworld.de/berlin-sehenswuerdigkeiten/], [snippet: Discover Berlin's culture, history, nightlife and nature with this guide to the top attractions. From Museum Island to the Be

In [8]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

Snippet: Berlin [bɛr'li:n] ist die Hauptstadt und ein Land der Bundesrepublik Deutschland. Die Großstadt ist mit rund 3,8 Millionen Einwohnern die bevölkerungsreichste und mit 892 Quadratkilometern die flächengrößte Gemeinde Deutschlands sowie die bevölkerungsreichste Stadt der Europäischen Union.
Title: Berlin - Wikipedia
Link: https://de.wikipedia.org/wiki/Berlin

--------------------------------------------------
Snippet: 31. Mai bis 02. Juni 2024: Das Berlin-Programm für das Wochenende in der Hauptstadt.
Title: Wochenend-Tipps: Endlich Wochenende in Berlin! - Berlin.de
Link: https://www.berlin.de/wochenend-tipps/

--------------------------------------------------
Snippet: Berlin in 7 Tagen - Die besten Berlin Sehenswürdigkeiten, Tipps, Highlights, Geheimtipps und Empfehlungen zu Hotels und Restaurants.
Title: Berlin Sehenswürdigkeiten: Die 41 schönsten Orte | in 3 - 7 Tagen
Link: https://wetraveltheworld.de/berlin-sehenswuerdigkeiten/

---------------------------------------------

### Wikipedia

In [9]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [10]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=10000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

'Page: NebulaGraph\nSummary: NebulaGraph is an open-source distributed graph database built for super large-scale graphs with milliseconds of latency. NebulaGraph adopts the Apache 2.0 license and  also  comes with a wide range of data visualization tools.\n\n'

In [11]:
output = wiki.invoke('Google Gemini')

In [12]:
print(output)

Page: Gemini (chatbot)
Summary: Gemini, formerly known as Bard, is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name and developed as a direct response to the meteoric rise of OpenAI's ChatGPT, it was launched in a limited capacity in March 2023 before expanding to other countries in May. It was previously based on PaLM, and initially the LaMDA family of large language models.
LaMDA had been developed and announced in 2021, but it was not released to the public out of an abundance of caution. OpenAI's launch of ChatGPT in November 2022 and its subsequent popularity caught Google executives off-guard and sent them into a panic, prompting a sweeping response in the ensuing months. After mobilizing its workforce, the company launched Bard in February 2023, which took center stage during the 2023 Google I/O keynote in May and was upgraded to the Gemini LLM in December. Bard and Duet AI were unified under the Gemini br

## Creating a ReAct Agent

In [13]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) 

True

In [14]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model (gpt-4-turbo-preview) with a temperature of 0. Utilize gpt-3.5-turbo if you use the free plan
llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)

# Define a template for answering questions
template = '''
Answer the following questions in GERMAN as best you can.
Questions: {q}
'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template)

# Pull the react prompt from the hub
prompt = hub.pull('hwchase17/react')

# displaying information about the react prompt
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)


# Create tools for the agent

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

# Combine the tools into a list
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)


In [15]:
# question = 'Generate the first 20 numbers in the Fibonacci series.'
# question = 'Who is the current prime minister of the UK?'
question = 'Tell me about Napoleon Bonaparte early life'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})




> Entering new AgentExecutor chain...
To answer this question in German, I will first find information about Napoleon Bonaparte's early life and then translate it into German.

Action: Wikipedia
Action Input: Napoleon BonapartePage: Napoleon
Summary: Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French emperor and military commander who rose to prominence during the French Revolution and led successful campaigns during the Revolutionary Wars. He was the leader of the French Republic as First Consul from 1799 to 1804, then of the French Empire as Emperor of the French from 1804 until 1814, and briefly again in 1815. His political and cultural legacy endures as a celebrated and controversial leader. He is considered one of the greatest military commanders in history and his wars and campaigns are still studied at military schools worldwide. However, historians still debate the degree to which he was resp

In [16]:
print(output['input'])


Answer the following questions in GERMAN as best you can.
Questions: Tell me about Napoleon Bonaparte early life



In [17]:
output['output']

'Napoleon Bonaparte wurde am 15. August 1769 auf der Insel Korsika geboren. Er unterstützte die Französische Revolution von 1789, während er in der französischen Armee diente, und versuchte, deren Ideale in seine Heimat Korsika zu tragen. Durch seine schnellen Erfolge in den militärischen Rängen, insbesondere nachdem er das regierende französische Direktorium durch das Feuern auf royalistische Aufständische gerettet hatte, stieg er rasch auf. Im Jahr 1796 begann er eine militärische Kampagne gegen die Österreicher und ihre italienischen Verbündeten, erzielte entscheidende Siege und wurde ein nationaler Held. Zwei Jahre später führte er eine militärische Expedition nach Ägypten durch, die als Sprungbrett zur politischen Macht diente. Er inszenierte einen Staatsstreich im November 1799 und wurde Erster Konsul der Republik. Im Jahr 1804, um seine Macht zu konsolidieren und zu erweitern, krönte er sich selbst zum Kaiser der Franzosen.'